In [104]:
%run '../_libtoolbox/init-notebook.py'
%run '../_libtoolbox/tbwidgets.py'
%run '../_libtoolbox/header.py'

<div style='position: fixed; bgcolor: white'>

&nbsp;[<img hspace="45%" align="middle" src="../_libtoolbox/nuflo-logo-N-512x512.png" alt="logo-N" width="8%" />](https://nuflo.de)

<center><h1>nuflo Engineering Toolbox</h1></center>

[nuflo is an engineering company](https://nuflo.de) that supports digitalization of engineering processes in fluid mechanics.

*The code for this tool can be found over at [github](https://github.com/JohK/engineering-toolbox).*  
*More interactive tools are located at [toolbox.nuflo.de](http://toolbox.nuflo.de).*

</div>

Boundary Layer Thickness on a Flat Plate
---
This is a tool for a quick estimation of the boundary layer thickness on a flat plate. For this estimation we use the momentum loss thickness based on two simple correlations:

The momentum loss thickness of a laminar boundary layer $\delta_l$ (Blasius solution):
$$ \delta_l = 0.664 \cdot \sqrt{\frac{\nu \cdot x}{u}} $$

For a turbulent flow:
$$ \delta_t = 0.0142 \cdot x \cdot \left(\frac{u\cdot x}{\nu}\right)^\left(-\frac{1}{7}\right) $$

Depending on the physical problem these may not be the right choice, if unsure ask an experienced engineer.

In [88]:
@np.vectorize
def momentum_boundarylayer(x, u, gas):
    rho = gas.density
    nu = gas.kinematic_viscosity
    
    fdelta_laminar = lambda x: 0.664*np.sqrt(nu*x/u)
    fdelta_turbulent = lambda x: 0.0142*(u*x/nu)**(-1/7)*x
    
    try:
        boundarylayer_thickness = [np.max(fdelta_laminar(xi), fdelta_turbulent(xi)) for xi in x]
    except TypeError:
        boundarylayer_thickness = np.max([fdelta_laminar(x), fdelta_turbulent(x)])
    
    # find where laminar and turbulent boundary layer thickness intersect
    sol = root_scalar(lambda x: fdelta_laminar(x)-fdelta_turbulent(x), bracket=[1e-6,10e6])
    if sol.converged:
        transition_position = sol.root
    else:
        transition_position = np.nan
    
    return(boundarylayer_thickness,
           transition_position)

In [89]:
def fplot(platelength, u, pressure, temperature, gasname):
    g = gas.state(gasname, 273.15+temperature, pressure)
    
    desc = '{:s}: T={:.1f}°C, p={:.3f}bar'.format(gasname, temperature, pressure/1e5)
    
    plt.figure()
    #print(xmax, u, gas.gasname)
    x = np.linspace(0.0000001, platelength, num=1000)
    
    if not np.isnan(g.density):
        bl, transition = momentum_boundarylayer(x, u, g)
        transition_bl, _ =  momentum_boundarylayer(transition[0], u, g)

        plt.plot(x, bl*1000, '-', label='boundary layer thickness')
        plt.plot(transition[0], transition_bl*1000, '.', label='laminar-turbulent transition')
        plt.xlim([0, platelength])
        plt.ylim([0, np.max(bl)*1000])
        plt.legend()
    else:
        plt.text(0.2, 0.5, 'Data not available, or\nnot a fluid in the property range',
                 transform=plt.gca().transAxes, 
                 bbox=dict(facecolor='red', alpha=0.2))
    plt.xlabel('length of flat plate in m')
    plt.ylabel('boundary layer thickness in mm')
    plt.grid()
    plt.title(desc)

    plt.show()

In [95]:
gas_dropdown = widgets.Dropdown(options=gas.state.fluids_list(), value='Air')

pwidget, pslider, _ = quantity('Pressure in Pa:', min=1, max=150e5, step=1e5, value=101325)
Twidget, Tslider, _ = quantity('Temperature in °C:', min=-30, max=55, step=1, value=20)

thermostate = widgets.VBox([
    pwidget,
    Twidget,
    widgets.HBox([widgets.Label('Gas / Specie Mixture:'), gas_dropdown])
])

platewidget, plateslider, _ = quantity('Platelength in m', min=0.001, max=10, step=0.001, value=0.1)
uwidget, uslider, _ = quantity('Velocity in m/s:', min=0.001, max=200, step=0.001, value=20)

flow = widgets.VBox([platewidget, uwidget])

accordion = widgets.Accordion(children=[thermostate, flow])
accordion.set_title(0, 'Thermodynamic State')
accordion.set_title(1, 'Flat Plate')


interactive_plot = interactive_output(fplot,
                               {'platelength': plateslider,
                                'u': uslider,
                                'pressure': pslider,
                                'temperature': Tslider,
                                'gasname': gas_dropdown})

display(accordion, interactive_plot)

Accordion(children=(VBox(children=(HBox(children=(Label(value='Pressure in Pa:', layout=Layout(flex='1 1 auto'…

Output()

In [96]:
from IPython.display import display, Markdown, Latex
display(Markdown('*some markdown* $\phi$'))

*some markdown* $\phi$